# Web Data Set Scraping in Python - Greenhouse Emissions Data

## Introduction

This project will focus on web scraping data from a webpage (Wikipedia) for use in an exploratory analysis between greenhouse emissions and GDP per capita by country. 

For this web scraping portion, we will utilize BeautifulSoup and Pandas dataframes to create, clean, and manipulate a data set from the internet. The original URL for the Wikipedia article can be found here:
https://en.wikipedia.org/wiki/List_of_countries_by_greenhouse_gas_emissions

### Import proper libraries

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

### Request and receive page information

In [2]:
# Send request to Wikipedia article
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_greenhouse_gas_emissions'
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    html_content = response.content
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


In [3]:
# Get Soup
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table')

In [10]:
# Initialize datasets and parse rows to append data into list
data = []
arrows = []

for row in table.find_all('tr'):
    cells = row.find_all(['td', 'th'])
    row_data = [cell.text.strip() for cell in cells]
    data.append(row_data)
    
    #creating data list to store unique arrow notation from original chart, essential to not lose data on trends
    img_tag = row.find('span', {'title': 'Positive decrease'}).find('img', 
                                                                    alt='Positive decrease'
                                                                   ) if row.find('span', 
                                                                                 {'title': 'Positive decrease'}
                                                                                )else None
    arrow = '▲' if img_tag and 'Decrease_Positive.svg' in img_tag['src'] else '▼' if img_tag and 'Increase_Positive.svg' in img_tag['src'] else ''
    arrows.append(arrow)

# Create a Pandas dataframe with new headers and data lists above
headers = ['Country/territory','GHG emissions 1970','GHG emissions 1990','GHG emissions 2005',
           'GHG emissions 2017','GHG emissions 2022','GHG per capita 2022', '% of world emissions 2022',
           '% Change from 1990']    
df = pd.DataFrame(data, columns=headers)

#reformat the percentages and format to numeric float values
df['% Change from 1990'] = pd.to_numeric(df['% Change from 1990'].str.replace(',', '').str.replace('%', '').replace('', '0'), errors='coerce')
df['% of world emissions 2022'] = pd.to_numeric(df['% of world emissions 2022'].str.replace('%', '').replace('', '0'), errors='coerce')



# Add temporary 'Color Arrow' column to the dataframe
df['Color Arrow'] = arrows

# Handle formatting of the '% Change from 1990' column, assign negative values for decrease in emissions
df.loc[df['Color Arrow'] == '▲', '% Change from 1990'] *= -1
df = df.drop('Color Arrow',axis=1)



### Clean and modify the dataframe 

In [5]:
# Original chart has cumulative categories in the first 5 rows, trim for later analysis
new_set = df.iloc[6:]

# View the first 20 rows of data to validate results with original webpage
new_set.iloc[:20]

,Country/territory,GHG emissions 1970,GHG emissions 1990,GHG emissions 2005,GHG emissions 2017,GHG emissions 2022,GHG per capita 2022,% of world emissions 2022,% Change from 1990
6,Aruba,45.2,214.4,462.6,467.2,496.7,4.64,0.001,231.7
7,Afghanistan,"17,336.2","13,775.6","18,191.3","31,773.0","29,117.9",0.73,0.054,211.4
8,Angola,"20,138.4","34,957.0","73,533.9","81,888.0","66,480.1",1.90,0.124,190.2
9,Anguilla,4.3,8.8,18.0,34.5,28.1,1.87,0.000,317.6
10,Albania,"8,261.3","11,568.1","8,070.1","9,281.3","7,983.4",2.71,0.015,-69.0
11,Netherlands Antilles,"15,078.6","2,855.9","6,146.2","4,171.5","2,154.5",13.06,0.004,-75.4
12,United Arab Emirates,"29,374.9","84,786.9","169,629.1","270,987.8","295,110.3",29.33,0.549,348.1
13,Argentina,"223,210.9","262,670.2","344,827.1","379,169.3","382,992.5",8.27,0.712,145.8
14,Armenia,"15,350.1","24,373.3","7,429.7","8,624.0","9,377.0",3.19,0.017,-38.5
15,Antigua and Barbuda,170.5,246.6,265.2,335.7,359.8,3.36,0.001,145.9


In [6]:
# Change data types from string to float
format_headers = headers[1:-2]
new_set[format_headers] = new_set[format_headers].replace({',': ''}, regex=True).astype(float)

/var/folders/t6/n0yl87jn4l5bzkmkpt6nbydr0000gn/T/ipykernel_4166/1329494446.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_set[format_headers] = new_set[format_headers].replace({',': ''}, regex=True).astype(float)


In [7]:
# Rename dataframe and use describe to inspect baseline trends and statistics
country_data = new_set
country_data.describe()

,GHG emissions 1970,GHG emissions 1990,GHG emissions 2005,GHG emissions 2017,GHG emissions 2022,GHG per capita 2022,% of world emissions 2022,% Change from 1990
count,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,208.000000,208.000000,208.000000
mean,1.151481e+05,1.567864e+05,1.984993e+05,2.396947e+05,2.529297e+05,6.975144,0.470236,204.021635
std,4.676741e+05,5.794117e+05,8.039335e+05,1.087207e+06,1.212423e+06,8.709528,2.254147,472.802651
min,3.800000e+00,7.000000e+00,1.140000e+01,1.860000e+01,2.030000e+01,0.590000,0.000000,-98.700000
25%,1.886250e+03,2.448900e+03,3.641300e+03,4.535600e+03,5.175450e+03,2.222500,0.009750,101.075000
50%,1.311140e+04,2.140295e+04,2.436560e+04,3.299650e+04,3.557850e+04,4.505000,0.066000,187.300000
75%,5.743785e+04,8.232773e+04,9.529588e+04,1.016419e+05,1.142168e+05,8.352500,0.212250,264.675000
max,5.750030e+06,6.163742e+06,8.431922e+06,1.371010e+07,1.568463e+07,67.380000,29.161000,6336.700000


### Export the data frame to csv to be used in SQL data exploration

In [8]:
# Export to csv without index 
country_data.to_csv('GHG_Emission_Data.csv', index=False)

## Conclusion and Recap

In this notebook, I successfully utilized BeautifulSoup to perform web scraping from the original wikipedia article. This new dataframe successfully captured all correct values from the original chart, even including percentage changes such as in the last column. 

This data can be used for visualizations and further analysis in Python, or exported to be used with SQL or Microsoft Excel. By web scraping our information, a new insightful dataset was easily formatted and made accessible for future analysis, without having to rely on existing databases. 

Please visit my corresponding SQL exploration and Power BI dashboard in my project portfolio, or at my personal portfolio website: www.johnsieve.com